In [1]:
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn import metrics

%run 00_Functions.ipynb
%run 01_Data_Cleaning.ipynb


Make a new version of a usable dataframe

In [15]:
X_3 = props.copy()

In [16]:
X_3.select_dtypes(include = np.number)

,Lot Frontage,Lot Area,Paved Street,Has All Utilities,Gentle Slope,Years since built,Years since remodel,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,...,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Years since last sold,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
109,NaN,13517,1,1,1,34,5,289.0,533.0,0.0,...,1,0,44,0,0,0,0,0,0,130500
544,43.0,11492,1,1,1,14,13,132.0,637.0,0.0,...,1,0,74,0,0,0,0,0,1,220000
153,68.0,7922,1,1,1,57,3,0.0,731.0,0.0,...,1,0,52,0,0,0,0,0,0,109000
318,73.0,9802,1,1,1,4,3,0.0,0.0,0.0,...,1,100,0,0,0,0,0,0,0,174000
255,82.0,14235,1,1,1,110,17,0.0,0.0,0.0,...,0,0,59,0,0,0,0,0,0,138500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,79.0,11449,1,1,1,3,3,0.0,1011.0,0.0,...,1,0,276,0,0,0,0,0,2,298751
785,NaN,12342,1,1,1,70,60,0.0,262.0,0.0,...,1,158,0,0,0,0,0,0,1,82500
916,57.0,7558,1,1,1,82,60,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,1,177000


### Dropping values that are either clearly colinear or clearly nonlinear

In [17]:
X_3.drop(['PID','Garage Yr Blt'], axis = 1, inplace = True)

In [18]:
non_numerics = X_3.select_dtypes(exclude = np.number)

### Exploring correlated numeric values to decide which to use in the model / engineer

In [19]:
corr = X_3.corr(numeric_only = True)[['SalePrice']].sort_values( by = 'SalePrice', ascending = False)

In [20]:
corr

,SalePrice
SalePrice,1.000000
Gr Liv Area,0.699026
Garage Area,0.648661
Garage Cars,0.647015
Total Bsmt SF,0.631975
1st Flr SF,0.623523
Full Bath,0.538471
Mas Vnr Area,0.512699
TotRms AbvGrd,0.505188
Fireplaces,0.471222


In [22]:
correlated_numeric_features = corr[(corr['SalePrice'] > 0.10) |  (corr['SalePrice'] < -0.10)].index[1:]

In [ ]:
X_3_nums = X_3.loc[:, list(correlated_numeric_features)]

## Creating dummies of categorical data

In [23]:
X_3_non_numerics = X_3.select_dtypes(exclude = np.number)

In [ ]:
replace_NaN_with_NA(X_3, [column for column in X_3_non_numerics.columns if X_3_non_numerics[column].isnull().sum() > 0])

In [26]:
X_3_dummy = pd.get_dummies(X_3_non_numerics, columns = non_numerics.columns, drop_first = True)

### Trimming off some dummy features that have underrepresented or overrepresented values

In [28]:
X_3_means = X_3_dummy.mean().sort_values().to_frame()
X_3_dummy_trimmed = list(X_3_means[(X_3_means[0] > 0.10) & (X_3_means[0] < 0.90)].index)

### Settling in on X and Y for this iteration

In [29]:
X_3_use = X_3_nums.join(X_3_dummy.loc[:, X_3_dummy_trimmed], on='Id', how='left')
y3 = X_3['SalePrice']

### Train test split

In [30]:
X3_train, X3_test, y3_train, y3_test = train_test_split(X_3_use, y3, test_size = 0.3, random_state=2023)

### We will be trying regularization here via Ridge cross validation

There is a not a huge case for this, given that previous OLS models were not overfit (in fact, they both performed better on the test data! But it is good to be dilignet

In [36]:
r_alphas = np.logspace(-2, 3, 100)

In [40]:
si = SimpleImputer()
sc = StandardScaler()
ridge_cv = RidgeCV(alphas = r_alphas, cv = 10)

In [41]:
X3_train_si = si.fit_transform(X3_train)
X3_test_si = si.transform(X3_test)

In [44]:
Z3_train = sc.fit_transform(X3_train_si)
Z3_test = sc.transform(X3_test_si)

In [45]:
ridge_cv.fit(Z3_train, y3_train)

RidgeCV(alphas=array([1.00000000e-02, 1.12332403e-02, 1.26185688e-02, 1.41747416e-02,
       1.59228279e-02, 1.78864953e-02, 2.00923300e-02, 2.25701972e-02,
       2.53536449e-02, 2.84803587e-02, 3.19926714e-02, 3.59381366e-02,
       4.03701726e-02, 4.53487851e-02, 5.09413801e-02, 5.72236766e-02,
       6.42807312e-02, 7.22080902e-02, 8.11130831e-02, 9.11162756e-02,
       1.02353102e-01, 1.14975700e-0...
       6.89261210e+01, 7.74263683e+01, 8.69749003e+01, 9.77009957e+01,
       1.09749877e+02, 1.23284674e+02, 1.38488637e+02, 1.55567614e+02,
       1.74752840e+02, 1.96304065e+02, 2.20513074e+02, 2.47707636e+02,
       2.78255940e+02, 3.12571585e+02, 3.51119173e+02, 3.94420606e+02,
       4.43062146e+02, 4.97702356e+02, 5.59081018e+02, 6.28029144e+02,
       7.05480231e+02, 7.92482898e+02, 8.90215085e+02, 1.00000000e+03]),
        cv=10)

In [46]:
ridge_cv.alpha_

312.5715849688235

In [47]:
ridge_cv.best_score_

0.7814200039675607